In [1]:
# coding: utf-8

### TP - Kafka lab

In [2]:
"""
    author: Loïc M. DIVAD
    date: 2016-10-20
    see also: ...
"""

'\n    author: Lo\xc3\xafc M. DIVAD\n    date: 2016-10-20\n    see also: ...\n'

In [15]:
import os
import sys
import json
import time
import logging
import urllib3
import random
import pandas as pd
from datetime import datetime
from optparse import OptionParser

In [4]:

# import re

In [5]:
# path mock
class optMock:
    def __init__(self):
        self.folder = '~/Github/ReportX/TP-kafka/data/'

In [6]:
# constant
METRICS = [
    "date_time", 
    "posa_continent", 
    "srch_destination_type_id", 
    "user_id", "is_mobile", 
    "channel", "is_booking"
]

LOG_FORMAT = '%(asctime)s [ %(levelname)s ] : %(message)s'

DATE_FORMAT = '%Y-%m-%d %X'

In [7]:
# functions

def to_date(t, DT=DATE_FORMAT):
    return datetime.strptime(y, DATE_FORMAT)

def waitingTime(y):
    return (y[1] - y[0]) / 1000

In [8]:
optparser = OptionParser()
optparser.add_option("-d", "--data", dest="folder", default="~/data/", help="path to the file train.csv")
#optparser.add_option("-l", "--docleft", dest="left", type="int", help="")

if 'ipykernel' not in sys.modules:
    options, args = optparser.parse_args()
else:
    options = optMock()

In [9]:
# loggin
logging.basicConfig(format=LOG_FORMAT)
log = logging.getLogger()
log.setLevel(logging.INFO)

In [10]:
if __name__ == "__main__":
    
    log.info("Loading the dataframe: %s"%options.folder)
    df = pd.DataFrame.from_csv(options.folder + '_train.csv', index_col=None)
    
    log.info("Sorting the dataframe by timestamp")
    df.sort(["date_time"], ascending=True, inplace=True)
    
    log.info("Sorting over. droping the index")
    df = df.reset_index(drop=True)
    
    log.info("dtypes date_time -> to datetime.datetime")
    df.date_time = df.date_time.apply(lambda y: datetime.strptime(y, DATE_FORMAT))
    
    start = df.date_time[0]
    print type(start)
    log.info("First date of logs : %s"%start)
    
    df["timestamp"] = df.date_time.apply(lambda y: (y - start).total_seconds())
    
    df["wait"] = pd.Series.rolling(df.timestamp, window=2).apply(lambda y: (y[1] - y[0]) / 10000 )
    
    df.wait.fillna(0.0, inplace=True)
    

<class 'pandas.tslib.Timestamp'>


/Users/loicmdivad/lmdadm/lib/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [11]:
#df[["date_time", "site_name", "posa_continent"]].head()
#df[["date_time", "timestamp", "site_name", "posa_continent"]].head()
df[["date_time", "timestamp", "wait", "site_name", "posa_continent"]].head()

,date_time,timestamp,wait,site_name,posa_continent
0,2013-01-10 18:04:37,0.0,0.0000,37,1
1,2013-01-10 18:24:37,1200.0,0.1200,37,1
2,2013-01-10 18:26:40,1323.0,0.0123,37,1
3,2013-01-10 18:30:08,1531.0,0.0208,37,1
4,2013-01-10 18:30:27,1550.0,0.0019,37,1


In [12]:
df.columns.values

array(['date_time', 'site_name', 'posa_continent', 'user_location_country',
       'user_location_region', 'user_location_city',
       'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
       'channel', 'srch_ci', 'srch_co', 'srch_adults_cnt',
       'srch_children_cnt', 'srch_rm_cnt', 'srch_destination_id',
       'srch_destination_type_id', 'is_booking', 'cnt', 'hotel_continent',
       'hotel_country', 'hotel_market', 'hotel_cluster', 'timestamp',
       'wait'], dtype=object)

In [13]:
df.posa_continent.unique()

array([1, 3, 2, 4])

In [14]:
for idx, row in df.iterrows():
    print row.wait
    time.sleep(row.wait)
    log.info(row[METRICS[:2]].to_dict())
    #if idx > 5:
    #    break

0.0
0.12
0.0123
0.0208
0.0019
6.7505
145.9778
0.0048
0.0045
156.432
8.1547


KeyboardInterrupt: 

70.63059463199927